## Aggregate site of crime APIs


  - https://rapidapi.com/collection/crime

## White Collar Crime Definition

Two post I found to figure out the definition to white collar crime and what constitute
  - https://ucr.fbi.gov/nibrs/nibrs_wcc.pdf
  - https://www.fbi.gov/investigate/white-collar-crime
  
## Project Presentation

Link to our project presentation
  - https://docs.google.com/presentation/d/1vRDEG75763FbuR4u8t4vXNta9lzOx1x-flDJ1Tv1bbM/edit?usp=sharing
  
## FBI Crime Data Explore github page

github page for fbi crime data frontend
  - https://github.com/fbi-cde/crime-data-frontend/blob/master/README.md
  
github page for fbi crime data api
  - https://github.com/fbi-cde/crime-data-api
  #### fbi crime data api support notes
    - https://github.com/fbi-cde/crime-data-api/blob/master/api_support_notes.md

In [1]:
# Installing the census library
#pip install census

In [2]:
# Dependencies
import numpy as np
import json
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time

from config import crimekey
from config import censuskey
from pprint import pprint
from census import Census

In [3]:
# API Setup
baseurl = "https://api.usa.gov/crime/fbi/sapi/"
c = Census(censuskey, year=2019)

In [45]:
# List of types of offenses for
offense = ["aggravated-assault", "burglary", "larceny", "motor-vehicle-theft", "homicide", "rape", "robbery", "arson", "violent-crime", "property-crime"]
offense_theft = ["burglary", "larceny", "motor-vehicle-theft", "robbery", "property-crime"]

# Empty list to store fbi crime url by theft offenses
fbi_crime_url_list = []

start_time = time.time()    # start timer

# For-loop to get fbi crime url by theft offenses for page = 0 only
for crime in offense_theft:
    fbi_crime_url = (f"{baseurl}api/summarized/state/CA/{crime}/2019/2019?page=0&API_KEY={crimekey}")
    fbi_crime_url_list.append(fbi_crime_url)
    
#pprint(fbi_crime_url_list)

end_time = time.time()     # end timer
time_diff = end_time - start_time    # time difference

# Prints runtime
#print(f"start time: {start_time}; end time: {end_time}; time diff: {time_diff}")

In [47]:
start_time = time.time()    # start timer

# Testing to see how API request pulls on burglary and first page of this offense
getdatasets = requests.get(fbi_crime_url_list[0])
datasets_json = getdatasets.json()
pprint(datasets_json)

end_time = time.time()    # end timer
time_diff = end_time - start_time    # time difference

# Prints runtime
print(f"start time: {start_time}; end time: {end_time}; time diff: {time_diff}")

{'pagination': {'count': 740, 'page': 0, 'pages': 37, 'per_page': 20},
 'results': [{'actual': 354,
              'cleared': 42,
              'data_range': None,
              'data_year': 2019,
              'offense': 'burglary',
              'ori': 'CA0010000',
              'state_abbr': 'CA'},
             {'actual': 218,
              'cleared': 23,
              'data_range': None,
              'data_year': 2019,
              'offense': 'burglary',
              'ori': 'CA0010100',
              'state_abbr': 'CA'},
             {'actual': 105,
              'cleared': 6,
              'data_range': None,
              'data_year': 2019,
              'offense': 'burglary',
              'ori': 'CA0010200',
              'state_abbr': 'CA'},
             {'actual': 771,
              'cleared': 119,
              'data_range': None,
              'data_year': 2019,
              'offense': 'burglary',
              'ori': 'CA0010300',
              'state_abbr': 'CA'},
     

In [29]:
# Find crime rate for each state
#stateAbbs = ['AL', 'AK', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS',
#            'KY', 'LA', 'MA', 'ME', 'MD', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 
#            'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

stateAbbs = ["CA"]

statedict = {}
record = 1

for page in range(datasets_json["pagination"]["pages"]):
    query_url = (f"https://api.usa.gov/crime/fbi/sapi/api/summarized/state/CA/{offense_theft[0]}/2019/2019?page={page}&API_KEY={crimekey}")
    
# loop through all states and pull data for each
for item in stateAbbs:
    #query_url = "https://api.usa.gov/crime/fbi/sapi/api/summarized/state/" + item + "/violent-crime/2019/2019?API_KEY=" + crimekey
    
    
    # Get response into JSON
    stateresponse = requests.get(query_url)
    statejson = stateresponse.json()
    x = len(statejson['results'])
    # Log each state and account for exceptions
    try:
        # Collect crime data and put it into a dictionary
        actuals = 0
        for i in range(x):
            actuals = actuals + statejson['results'][i]['actual']
            statedict[item] = [actuals]
        print(f"Processing record {record} | {item}")
        record += 1
        
    # Exception if state data isn't found
    except:
        print(f"Data not found for {item}. Skipping...")

print("----------Job complete!----------")
pprint(query_url)


for i in 


Processing record 1 | CA
----------Job complete!----------
'https://api.usa.gov/crime/fbi/sapi/api/summarized/state/CA/burglary/2019/2019?API_KEY=ynex5V8kmuU8hhlhdljbjVB6Ih5LYjBvWIStnRUD'


In [30]:
# Convert the dictionary into a dataframe
statedf = pd.DataFrame.from_dict(statedict, orient='index')
statedf

,0
CA,6516


In [ ]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B19301_001E", "B17001_002E", "B23025_005E"), {'for': 'state:*'})

#census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
#                          "B19301_001E",
#                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()